In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.model_selection import train_test_split

In [4]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

In [5]:
x_train=tf.image.resize(x_train,(128,128))
x_test=tf.image.resize(x_test,(128,128))

In [6]:
x_train=x_train.numpy()
x_test=x_test.numpy()

In [7]:
x_train=tf.keras.applications.mobilenet.preprocess_input(x_train)
x_test=tf.keras.applications.mobilenet.preprocess_input(x_test)

In [8]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [9]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [10]:
base_model=MobileNetV3Large(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3)
)

/usr/local/lib/python3.10/dist-packages/keras/src/applications/mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


In [11]:
base_model.trainable=False

In [12]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Large (Functional)        │ (None, 4, 4, 960)           │       2,996,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 960)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         984,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,669,412 (17.81 MB)

 Trainable params: 1,669,476 (6.37 MB)

 Non-trainable params: 2,999,936 (11.44 MB)

In [13]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger
early_stopper=EarlyStopping(patience=5,monitor="val_loss",restore_best_weights=True)

In [15]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [16]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [17]:
import time
start_time=time.time()

In [18]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 157s 129ms/step - accuracy: 0.0199 - loss: 5.1891 - val_accuracy: 0.0826 - val_loss: 4.1895
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 136s 114ms/step - accuracy: 0.0402 - loss: 4.6443 - val_accuracy: 0.0974 - val_loss: 4.0475
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 133s 114ms/step - accuracy: 0.0556 - loss: 4.4405 - val_accuracy: 0.1154 - val_loss: 3.9320
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.0662 - loss: 4.3244 - val_accuracy: 0.1352 - val_loss: 3.8059
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.0723 - loss: 4.2369 - val_accuracy: 0.1434 - val_loss: 3.7537
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - accuracy: 0.0821 - loss: 4.1415 - val_accuracy: 0.1462 - val_loss: 3.7268
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 133s 114ms/step - accuracy: 0.0911 - loss: 4.0671 - val_accuracy: 0.1617 - val_loss: 3.6375
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 134s 114ms/step - ac

In [19]:
head_training_time=time.time()-start_time

In [20]:
base_model.trainable=True

In [21]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [22]:
start_fine_tune=time.time()

In [23]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 227s 176ms/step - accuracy: 0.0264 - loss: 4.9649 - val_accuracy: 0.0171 - val_loss: 5.3466
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 175s 139ms/step - accuracy: 0.0987 - loss: 3.8263 - val_accuracy: 0.0275 - val_loss: 5.2827
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 163s 139ms/step - accuracy: 0.1588 - loss: 3.4320 - val_accuracy: 0.0383 - val_loss: 5.2379
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 163s 139ms/step - accuracy: 0.1977 - loss: 3.2071 - val_accuracy: 0.0651 - val_loss: 4.5411
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 163s 139ms/step - accuracy: 0.2405 - loss: 2.9817 - val_accuracy: 0.1814 - val_loss: 3.4009


In [24]:
fine_tune_time=time.time()-start_fine_tune

In [25]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step


In [26]:
test_loss,test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.0167 - loss: 5.3480

Test Accuracy: 1.76%
Total training time: 3594.83 seconds


In [27]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0160 - loss: 5.3564
Validation accuracy: 1.7120%


In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred_class)
precision = precision_score(y_test, y_pred_class, average='weighted')
recall = recall_score(y_test, y_pred_class, average='weighted')
f1 = f1_score(y_test, y_pred_class, average='weighted')

print("\nPerformance Metrics Breakdown for MobileNetV3Large:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for MobileNetV3Large:
Accuracy: 0.0176
Precision: 0.0053
Recall: 0.0176
F1-Score: 0.0056


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["MobileNetV3 Large"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [30]:
print(results)

               Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  MobileNetV3 Large         0.0176              0.01712   5.347051   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             2701.093899            893.731918       4669412  


In [31]:
results.to_csv("MobileNetV3Large.csv",index=False)
print("\nResults saved to MobileNetV3Large.csv")


Results saved to MobileNetV3Large.csv
